In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
df=pd.read_csv('Churn_Modelling.csv' )

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
#removing irrelevent column
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe_geography=OneHotEncoder()
ohe_value=ohe_geography.fit_transform(df[['Geography']])
ohe_value.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
df_encoded=pd.DataFrame(ohe_value.toarray(),columns=ohe_geography.get_feature_names_out())

In [12]:
df_encoded.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [13]:
df=pd.concat([df.drop('Geography',axis=1),df_encoded],axis=1)

In [14]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
#save the encoders and Standard scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('OHE_Geography.pkl','wb') as file:
    pickle.dump(ohe_geography,file)

In [16]:
#Dividing data into dependent and independent faetures
x=df.drop('Exited',axis=1)
y=df['Exited']

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [18]:
#standard Scaler
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
x_train=std_scaler.fit_transform(x_train)
x_test=std_scaler.transform(x_test)
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [19]:
with open("std_scaler.pkl",'wb') as file:
    pickle.dump(std_scaler,file)

ANN Implementation

In [20]:
import tensorflow as tf

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

In [22]:
x_train.shape[1]

12

In [23]:
#Building ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),#HL1 connected with input layer
    Dense(32,activation='relu'),#hl2
    Dense(1,activation='sigmoid')#output layer
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()


In [26]:
# We can use optimizer and loss function in this way also but it takes defalut value of alpha and loss
# model.compile(optimizer='Adam',loss='BinaryCrossEntropy',metrics=['accuracy'])

In [27]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [28]:
#setting the tensorboard

from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
log_dir='logs/fit/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [29]:
#setup Early Stopping
early_Stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [30]:
#training MOdel
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_Stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 2ms/step - loss: 0.3970 - accuracy: 0.8316 - val_loss: 0.3649 - val_accuracy: 0.8500
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3534 - accuracy: 0.8555 - val_loss: 0.3712 - val_accuracy: 0.8505
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3512 - accuracy: 0.8579 - val_loss: 0.3629 - val_accuracy: 0.8415
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3436 - accuracy: 0.8596 - val_loss: 0.3573 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3434 - accuracy: 0.8585 - val_loss: 0.3453 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3390 - accuracy: 0.8631 - val_loss: 0.3485 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3368 - accuracy: 0.8636 - val_loss: 0.3400 - val_accuracy: 0.85

In [31]:
model.save('model.h5')

d:\Father's_Document\code\Projects\CustomerChurnPredcition\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
#load tensorboard Extension
%load_ext tensorboard


In [33]:
%tensorboard --logdir logs/fit